<a href="https://colab.research.google.com/github/yonabell/Report/blob/main/All_reporting_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
from google.colab import files
uploaded = files.upload()

Saving 2024_sample_data.xlsx to 2024_sample_data.xlsx


In [3]:
# Load the data with no predefined header
file_path = '2024_sample_data.xlsx'
cpt_df = pd.read_excel(file_path, engine='openpyxl', header=4)

In [4]:
# Filtering rows and extracting only 'A' followed by nine digits in one step
filtered_cpt_df = cpt_df[cpt_df['Unnamed: 0'].str.extract(r'(A\d{9})', expand=False).notna()].copy()


In [5]:
# Assuming your DataFrame is named 'filtered_cpt_df'
filtered_cpt_df['ID'] = filtered_cpt_df['Unnamed: 0'].str.extract(r'(A\d{9})', expand=False)


In [6]:
# Reorder columns to place 'ID' next to 'Unnamed: 0'
cols = filtered_cpt_df.columns.tolist()
id_index = cols.index('ID')
cols = cols[:1] + [cols[id_index]] + cols[1:id_index] + cols[id_index+1:]
filtered_cpt_df = filtered_cpt_df[cols]


In [7]:
# Assuming your DataFrame is named 'filtered_cpt_df'
filtered_cpt_df = filtered_cpt_df.drop('Unnamed: 0', axis=1)


In [8]:
# Assuming your DataFrame is named 'filtered_cpt_df'
filtered_cpt_df = filtered_cpt_df.drop(filtered_cpt_df.columns[1:8], axis=1)


In [9]:
# Assuming your DataFrame is named 'filtered_cpt_df'
column_names = filtered_cpt_df.columns.tolist()


In [10]:
# Select every 7th column starting from the 8th column (index 7 in Python)
columns_to_drop = column_names[7::7]

In [11]:
# Drop the selected columns from the DataFrame
filtered_cpt_df.drop(columns=columns_to_drop, inplace=True)

In [12]:
filtered_cpt_df

,ID,Purch. Volume.1,S-Revenues.1,Profit Margin I.1,PM I % of Purchased Volume.1,Profit Margin I*.1,PM I* % of Purchased Volume.1,Purch. Volume.2,S-Revenues.2,Profit Margin I.2,PM I % of Purchased Volume.2,Profit Margin I*.2,PM I* % of Purchased Volume.2,Purch. Volume.3,S-Revenues.3,Profit Margin I.3,PM I % of Purchased Volume.3,Profit Margin I*.3,PM I* % of Purchased Volume.3
5,A987654321,678.0,87464.0,7656.0,44.68,454.0,0.09,4355.0,43.0,2.0,0.14,546.0,3.785323,65.0,34.0,34.0,9.8384,45.0,2.033
8,A123456789,8876.0,3566.0,765.0,71.44,4326.0,0.12,1345.0,33.0,32.0,0.20,65.0,5.724400,543.0,725.0,643.0,15.7322,542.0,2.519


In [13]:
# Assuming your DataFrame is named 'filtered_cpt_df' and it's already loaded.

# Columns to be aggregated (excluding 'ID')
agg_cols = ['Purch. Volume', 'S-Revenues', 'Profit Margin I', 'PM I % of Purchased Volume', 'Profit Margin I*', 'PM I* % of Purchased Volume']

# Get all columns that start with the names in agg_cols
cols_to_convert = [col for col in filtered_cpt_df.columns if any(col.startswith(agg_col) for agg_col in agg_cols)]

# Convert the selected columns to numeric
filtered_cpt_df[cols_to_convert] = filtered_cpt_df[cols_to_convert].apply(pd.to_numeric, errors='coerce')

# Create a new DataFrame to store the aggregated results
aggregated_df = pd.DataFrame(columns=['ID'] + agg_cols)

# Group the DataFrame by 'ID' and aggregate
for id_val, group in filtered_cpt_df.groupby('ID'):
    new_row = {'ID': id_val}
    for agg_col in agg_cols:
        new_row[agg_col] = group[[col for col in group.columns if col.startswith(agg_col)]].sum(axis=1).sum()  # Sum across rows and then sum the results
    aggregated_df = pd.concat([aggregated_df, pd.DataFrame([new_row])], ignore_index=True)

# Display the aggregated DataFrame
aggregated_df

<ipython-input-13-5a282a9a8212>:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  aggregated_df = pd.concat([aggregated_df, pd.DataFrame([new_row])], ignore_index=True)


,ID,Purch. Volume,S-Revenues,Profit Margin I,PM I % of Purchased Volume,Profit Margin I*,PM I* % of Purchased Volume
0,A123456789,10764.0,4324.0,6373.0,87.3722,4933.0,8.363400
1,A987654321,5098.0,87541.0,8737.0,54.6584,1045.0,5.908323


In [19]:
# Rename columns
new_column_names = {
    'ID': 'BPID',
    'Purch. Volume': 'PurchVolume',
    'S-Revenues': 'Revenue',
    'Profit Margin I': 'PMI',
    'PM I % of Purchased Volume': 'PMIRate',
    'Profit Margin I*': 'PMIStar',
    'PM I* % of Purchased Volume': 'PMIStarRate'
}
aggregated_df = aggregated_df.rename(columns=new_column_names)

# Display the aggregated DataFrame
aggregated_df



,BPID,PurchVolume,Revenue,PMI,PMIRate,PMIStar,PMIStarRate
0,A123456789,10764.0,4324.0,6373.0,87.3722,4933.0,8.363400
1,A987654321,5098.0,87541.0,8737.0,54.6584,1045.0,5.908323


In [24]:
import os
import pandas as pd

# ... (Your existing code for data processing) ...
# aggregated_df = aggregated_df.rename(columns=new_column_names)

# Specify the folder path where you want to save the file
folder_path = r'G:\report\out_put'  # Replace with your actual folder path

# Ensure the folder exists; create it if it doesn't
#if not os.path.exists(folder_path):
#    os.makedirs(folder_path)

# Save the DataFrame to an Excel file in the specified folder
file_name = 'aggregated_data.xlsx'
file_path = os.path.join(folder_path, file_name)
aggregated_df.to_excel(file_path, index=False) # Save the file using to_excel


In [25]:
# prompt: i want to download this aggregated_df right now to my pc

# ... (Your existing code for data processing) ...
# aggregated_df = aggregated_df.rename(columns=new_column_names)

# Save the DataFrame to an Excel file
file_name = 'aggregated_data.xlsx'
aggregated_df.to_excel(file_name, index=False)

# Download the file
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [26]:
from google.colab import files
uploaded = files.upload()

# Load the data with no predefined header
file_path = '2023_sample_data.xlsx'

def clean_data(file_path):
    # Load the data with no predefined header
    cpt_df = pd.read_excel(file_path, engine='openpyxl', header=4)

    # Filtering rows and extracting only 'A' followed by nine digits
    filtered_cpt_df = cpt_df[cpt_df['Unnamed: 0'].str.extract(r'(A\d{9})', expand=False).notna()].copy()

    # Extracting the ID
    filtered_cpt_df['ID'] = filtered_cpt_df['Unnamed: 0'].str.extract(r'(A\d{9})', expand=False)

    # Reorder columns to place 'ID' next to 'Unnamed: 0'
    cols = filtered_cpt_df.columns.tolist()
    id_index = cols.index('ID')
    cols = cols[:1] + [cols[id_index]] + cols[1:id_index] + cols[id_index + 1:]
    filtered_cpt_df = filtered_cpt_df[cols]

    # Drop 'Unnamed: 0' column
    filtered_cpt_df = filtered_cpt_df.drop('Unnamed: 0', axis=1)

    # Drop specific columns (1 to 7)
    filtered_cpt_df = filtered_cpt_df.drop(filtered_cpt_df.columns[1:8], axis=1)

    # Select every 7th column starting from the 8th column (index 7)
    column_names = filtered_cpt_df.columns.tolist()
    columns_to_drop = column_names[7::7]

    # Drop the selected columns from the DataFrame
    filtered_cpt_df.drop(columns=columns_to_drop, inplace=True)

    return filtered_cpt_df  # Ensure this returns filtered_cpt_df for consistency


Saving 2023_sample_data.xlsx to 2023_sample_data.xlsx


In [27]:
# Call the clean_data function
filtered_cpt_df = clean_data(file_path)

In [28]:
filtered_cpt_df

,ID,Purch. Volume.1,S-Revenues.1,Profit Margin I.1,PM I % of Purchased Volume.1,Profit Margin I*.1,PM I* % of Purchased Volume.1,Purch. Volume.2,S-Revenues.2,Profit Margin I.2,PM I % of Purchased Volume.2,Profit Margin I*.2,PM I* % of Purchased Volume.2,Purch. Volume.3,S-Revenues.3,Profit Margin I.3,PM I % of Purchased Volume.3,Profit Margin I*.3,PM I* % of Purchased Volume.3
5,A987654321,678.0,87464.0,7656.0,44.68,454.0,0.09,4355.0,43.0,2.0,0.14,546.0,3.785323,65.0,34.0,34.0,9.8384,45.0,2.033
8,A123456789,8876.0,3566.0,765.0,71.44,4326.0,0.12,1345.0,33.0,32.0,0.20,65.0,5.724400,543.0,725.0,643.0,15.7322,542.0,2.519


In [30]:
# Assuming your DataFrame is named 'filtered_cpt_df' and it's already loaded.

# Create a dictionary to store the month-wise column mappings
month_columns = {}
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

for i, month in enumerate(months):
  start_col_index = 1 + (i * 6)  # Starting column index for the month
  end_col_index = start_col_index + 6
  month_columns[month] = filtered_cpt_df.columns[start_col_index:end_col_index].tolist()

In [31]:
# prompt: Now i want to filter the data as follow, of cour there will be the ID column and then the months feburary and march. And i want the months to filter to be dynamic as I will need different months each time i use this code, therfore maybe assign it to a variable called filltered month. Then the out put will be the first column(id) and then the columns for the fillered months. and these filltered months should be daynamic, in a way that i can change it to the months i want. not always feburary and march

# ... (Your existing code for data processing) ...

# Define the months you want to filter for (dynamically)
filtered_months = ['February', 'March']  # Example: Change this list as needed

# Create a list to store the selected columns
selected_columns = ['ID']

# Loop through the filtered months and add corresponding columns to the list
for month in filtered_months:
  if month in month_columns:
    selected_columns.extend(month_columns[month])

# Filter the DataFrame with selected columns
filtered_df = filtered_cpt_df[selected_columns]

# Display the filtered DataFrame
filtered_df

# ... (Your remaining code for further processing) ...

,ID,Purch. Volume.2,S-Revenues.2,Profit Margin I.2,PM I % of Purchased Volume.2,Profit Margin I*.2,PM I* % of Purchased Volume.2,Purch. Volume.3,S-Revenues.3,Profit Margin I.3,PM I % of Purchased Volume.3,Profit Margin I*.3,PM I* % of Purchased Volume.3
5,A987654321,4355.0,43.0,2.0,0.14,546.0,3.785323,65.0,34.0,34.0,9.8384,45.0,2.033
8,A123456789,1345.0,33.0,32.0,0.20,65.0,5.724400,543.0,725.0,643.0,15.7322,542.0,2.519
